In [1]:
import os
import torch
import gym
import numpy as np
from TD3_torch.TD3 import TD3
from PIL import Image
from TD3_torch.utils import ReplayBuffer

env_name = 'BipedalWalkerHardcore-v2'
learning_rate_base = 0.0001
log_interval = 10           # print avg reward after interval
random_seed = 1
gamma = 0.99                # discount for future rewards
batch_size = 256          # num of transitions sampled from replay buffer
exploration_noise = 0.3 
polyak_int = [0.9999, 0.999999]              # target policy update parameter (1-tau)
policy_noise = 0.2          # target policy smoothing noise
noise_clip = 0.5
policy_delay = 2            # delayed policy updates parameter
max_episodes = 100000         # max num of episodes
max_timesteps = 3000        # max timesteps in one episode
max_buffer_length = 2000000
directory = "./preTrained/td3_torch/{}".format(env_name) # save trained models
filename = "TD3_torch_{}_{}".format(env_name, random_seed)
reward_history = []



def train():
    env = gym.make(env_name)
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.shape[0]
    max_action = float(env.action_space.high[0])
    polyak = polyak_int[0]
    
    policy = TD3(state_dim, action_dim, max_action, lr=learning_rate_base)
    replay_buffer = ReplayBuffer(max_length=max_buffer_length)
    
    print("action_space={}".format(env.action_space))
    print("obs_space={}".format(env.observation_space))
    print("threshold={} \n".format(env.spec.reward_threshold))
    
    if random_seed:
        print("Random Seed: {}".format(random_seed))
        env.seed(random_seed)
        torch.manual_seed(random_seed)
        np.random.seed(random_seed)
    
    # logging variables:        
    log_f = open("log.txt","w+")
    
    # training procedure:
    for episode in range(1, max_episodes+1):
        ep_reward = 0
        state = env.reset()
       
        for t in range(max_timesteps):
            # select action and add exploration noise:
            action = policy.select_action(state)
            action = action + np.random.normal(0, exploration_noise, size=env.action_space.shape[0])
            action = action.clip(env.action_space.low, env.action_space.high)
            
            # take action in env:
            next_state, reward, done, _ = env.step(action)
            replay_buffer.add((state, action, reward, next_state, float(done)))
            state = next_state
            
            ep_reward += reward
            
            # if episode is done then update policy:
            if done or t==(max_timesteps-1):
                policy.update(replay_buffer, t, batch_size, gamma, polyak, policy_noise, noise_clip, policy_delay)
                break
        
        reward_history.append(ep_reward)
        avg_reward = np.mean(reward_history[-100:]) 
        
        # logging updates:        
        log_f.write('{},{}\n'.format(episode, ep_reward))
        log_f.flush()
       
        
        # if avg reward > 300 then save and stop traning:
        if avg_reward >= env.spec.reward_threshold: 
            print("########## Solved! ###########")
            name = filename + '_solved'
            policy.save(directory, name)
            log_f.close()
            break
            
        # Calculate polyak
        #part = (env.spec.reward_threshold - avg_reward) / (env.spec.reward_threshold + 150)
        #if part > 1:
        #    part = 1
        #polyak = polyak_int[0] + (1 - part) * (polyak_int[1] - polyak_int[0])     
        
        # Calculate LR
        part = (env.spec.reward_threshold - avg_reward) / (env.spec.reward_threshold + 150)
        if part > 1:
            part = 1
        learning_rate = learning_rate_base - learning_rate_base * (1 - part) * 0.9
        policy.set_optimizers(lr=learning_rate)
        
        
        if episode > 500:
            policy.save(directory, filename)
        
        # print avg reward every log interval:
        if episode % log_interval == 0:            
            print("Ep: {}   Rew: {:3.2f}   Avg Rew: {:3.2f}   LR: {:8.8f}   Polyak: {:6.5f}   Bf: {:2.0f}   Loss: {:5.3f}  {:5.3f}  {:5.3f}".format(
                episode, ep_reward, avg_reward, learning_rate, polyak, replay_buffer.get_fill(), policy.actor_loss, policy.loss_Q1, policy.loss_Q2))

train()


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
action_space=Box(4,)
obs_space=Box(24,)
threshold=300 

Random Seed: 1
Ep: 10   Rew: -100.77   Avg Rew: -97.37   LR: 0.00008947   Polyak: 0.99990   Bf:  0   Loss: -0.151  0.056  0.052
Ep: 20   Rew: -100.98   Avg Rew: -100.14   LR: 0.00009003   Polyak: 0.99990   Bf:  0   Loss: 0.036  0.552  0.585
Ep: 30   Rew: -101.00   Avg Rew: -100.81   LR: 0.00009016   Polyak: 0.99990   Bf:  0   Loss: -0.114  30.415  28.746
Ep: 40   Rew: -100.70   Avg Rew: -101.19   LR: 0.00009024   Polyak: 0.99990   Bf:  0   Loss: -0.295  3.059  3.283
Ep: 50   Rew: -100.10   Avg Rew: -103.14   LR: 0.00009063   Polyak: 0.99990   Bf:  1   Loss: -0.355  0.251  0.203
Ep: 60   Rew: -115.28   Avg Rew: -103.21   LR: 0.00009064   Polyak: 0.99990   Bf:  1   Loss: -0.358  3.787  5.897
Ep: 70   Rew: -103.69   Avg Rew: -104.12   LR: 

Ep: 710   Rew: -104.16   Avg Rew: -108.61   LR: 0.00009172   Polyak: 0.99990   Bf:  4   Loss: 1.478  1.705  1.128
Ep: 720   Rew: -114.11   Avg Rew: -108.25   LR: 0.00009165   Polyak: 0.99990   Bf:  4   Loss: 3.338  1.888  2.007
Ep: 730   Rew: -115.58   Avg Rew: -108.06   LR: 0.00009161   Polyak: 0.99990   Bf:  4   Loss: 2.028  0.921  1.665
Ep: 740   Rew: -110.49   Avg Rew: -107.45   LR: 0.00009149   Polyak: 0.99990   Bf:  4   Loss: 4.080  1.202  1.624
Ep: 750   Rew: -114.17   Avg Rew: -107.85   LR: 0.00009157   Polyak: 0.99990   Bf:  4   Loss: 2.864  3.422  2.777
Ep: 760   Rew: -96.89   Avg Rew: -108.93   LR: 0.00009179   Polyak: 0.99990   Bf:  4   Loss: 0.998  1.716  1.876
Ep: 770   Rew: -111.15   Avg Rew: -108.14   LR: 0.00009163   Polyak: 0.99990   Bf:  4   Loss: 2.309  0.538  0.618
Ep: 780   Rew: -112.56   Avg Rew: -112.07   LR: 0.00009241   Polyak: 0.99990   Bf:  5   Loss: 1.453  1.454  1.489
Ep: 790   Rew: -103.70   Avg Rew: -111.84   LR: 0.00009237   Polyak: 0.99990   Bf:  5   L

Ep: 1430   Rew: -108.77   Avg Rew: -122.49   LR: 0.00009450   Polyak: 0.99990   Bf: 13   Loss: 7.220  1.588  1.035
Ep: 1440   Rew: -126.69   Avg Rew: -121.78   LR: 0.00009436   Polyak: 0.99990   Bf: 13   Loss: 5.948  1.291  0.870
Ep: 1450   Rew: -139.66   Avg Rew: -120.39   LR: 0.00009408   Polyak: 0.99990   Bf: 13   Loss: 5.617  1.860  2.138
Ep: 1460   Rew: -108.50   Avg Rew: -119.16   LR: 0.00009383   Polyak: 0.99990   Bf: 13   Loss: 5.384  0.519  0.747
Ep: 1470   Rew: -116.78   Avg Rew: -118.15   LR: 0.00009363   Polyak: 0.99990   Bf: 13   Loss: 4.270  0.533  0.645
Ep: 1480   Rew: -112.68   Avg Rew: -116.19   LR: 0.00009324   Polyak: 0.99990   Bf: 13   Loss: 4.537  1.216  0.981
Ep: 1490   Rew: -111.13   Avg Rew: -115.73   LR: 0.00009315   Polyak: 0.99990   Bf: 13   Loss: 5.569  0.761  0.731
Ep: 1500   Rew: -103.12   Avg Rew: -113.71   LR: 0.00009274   Polyak: 0.99990   Bf: 13   Loss: 5.077  0.995  0.651
Ep: 1510   Rew: -116.69   Avg Rew: -113.35   LR: 0.00009267   Polyak: 0.99990   

Ep: 2150   Rew: -105.16   Avg Rew: -108.26   LR: 0.00009165   Polyak: 0.99990   Bf: 17   Loss: 9.196  2.119  1.226
Ep: 2160   Rew: -103.52   Avg Rew: -107.89   LR: 0.00009158   Polyak: 0.99990   Bf: 17   Loss: 9.454  1.634  1.499
Ep: 2170   Rew: -102.74   Avg Rew: -107.34   LR: 0.00009147   Polyak: 0.99990   Bf: 17   Loss: 8.574  1.430  1.798
Ep: 2180   Rew: -106.93   Avg Rew: -106.36   LR: 0.00009127   Polyak: 0.99990   Bf: 17   Loss: 5.789  1.153  1.029
Ep: 2190   Rew: -99.27   Avg Rew: -105.04   LR: 0.00009101   Polyak: 0.99990   Bf: 17   Loss: 9.414  1.891  1.203
Ep: 2200   Rew: -97.01   Avg Rew: -105.06   LR: 0.00009101   Polyak: 0.99990   Bf: 17   Loss: 5.579  1.281  0.700
Ep: 2210   Rew: -110.49   Avg Rew: -105.90   LR: 0.00009118   Polyak: 0.99990   Bf: 17   Loss: 7.219  1.075  1.059
Ep: 2220   Rew: -96.34   Avg Rew: -105.83   LR: 0.00009117   Polyak: 0.99990   Bf: 17   Loss: 8.190  2.511  2.479
Ep: 2230   Rew: -131.53   Avg Rew: -106.11   LR: 0.00009122   Polyak: 0.99990   Bf:

Ep: 2870   Rew: -115.73   Avg Rew: -102.07   LR: 0.00009041   Polyak: 0.99990   Bf: 20   Loss: 10.822  1.749  1.115
Ep: 2880   Rew: -116.74   Avg Rew: -102.64   LR: 0.00009053   Polyak: 0.99990   Bf: 20   Loss: 10.808  1.343  1.793
Ep: 2890   Rew: -102.07   Avg Rew: -102.93   LR: 0.00009059   Polyak: 0.99990   Bf: 20   Loss: 11.712  1.678  1.345
Ep: 2900   Rew: -97.08   Avg Rew: -102.75   LR: 0.00009055   Polyak: 0.99990   Bf: 20   Loss: 8.206  1.201  1.423
Ep: 2910   Rew: -99.40   Avg Rew: -103.35   LR: 0.00009067   Polyak: 0.99990   Bf: 21   Loss: 9.343  2.011  2.121
Ep: 2920   Rew: -97.12   Avg Rew: -103.69   LR: 0.00009074   Polyak: 0.99990   Bf: 21   Loss: 10.843  1.290  1.494
Ep: 2930   Rew: -117.88   Avg Rew: -103.49   LR: 0.00009070   Polyak: 0.99990   Bf: 21   Loss: 12.355  6.661  5.475
Ep: 2940   Rew: -120.01   Avg Rew: -104.01   LR: 0.00009080   Polyak: 0.99990   Bf: 21   Loss: 9.122  4.189  4.237
Ep: 2950   Rew: -95.36   Avg Rew: -103.77   LR: 0.00009075   Polyak: 0.99990  

...

Ep: 23230   Rew: -31.00   Avg Rew: -2.73   LR: 0.00007055   Polyak: 0.99990   Bf: 100   Loss: 6.514  0.957  1.205
Ep: 23240   Rew: -21.58   Avg Rew: -4.65   LR: 0.00007093   Polyak: 0.99990   Bf: 100   Loss: 6.483  0.980  0.834
Ep: 23250   Rew: -46.43   Avg Rew: 1.57   LR: 0.00006969   Polyak: 0.99990   Bf: 100   Loss: 6.394  1.562  0.724
Ep: 23260   Rew: -29.61   Avg Rew: 8.95   LR: 0.00006821   Polyak: 0.99990   Bf: 100   Loss: 6.867  0.799  0.932
Ep: 23270   Rew: 9.88   Avg Rew: 6.84   LR: 0.00006863   Polyak: 0.99990   Bf: 100   Loss: 6.225  1.957  2.446
Ep: 23280   Rew: -44.09   Avg Rew: 4.07   LR: 0.00006919   Polyak: 0.99990   Bf: 100   Loss: 6.132  0.621  0.739
Ep: 23290   Rew: -90.58   Avg Rew: 2.58   LR: 0.00006948   Polyak: 0.99990   Bf: 100   Loss: 6.548  0.670  0.367
Ep: 23300   Rew: -106.64   Avg Rew: 3.72   LR: 0.00006926   Polyak: 0.99990   Bf: 100   Loss: 6.602  1.502  0.715
Ep: 23310   Rew: -15.19   Avg Rew: 5.92   LR: 0.00006882   Polyak: 0.99990   Bf: 100   Loss: 5.

Ep: 23960   Rew: -94.97   Avg Rew: -7.69   LR: 0.00007154   Polyak: 0.99990   Bf: 100   Loss: 5.887  0.470  1.290
Ep: 23970   Rew: -54.11   Avg Rew: -13.28   LR: 0.00007266   Polyak: 0.99990   Bf: 100   Loss: 5.778  0.277  0.367
Ep: 23980   Rew: -49.21   Avg Rew: -22.97   LR: 0.00007459   Polyak: 0.99990   Bf: 100   Loss: 6.548  0.628  0.699
Ep: 23990   Rew: -76.53   Avg Rew: -24.96   LR: 0.00007499   Polyak: 0.99990   Bf: 100   Loss: 5.784  0.337  0.333
Ep: 24000   Rew: -82.22   Avg Rew: -18.16   LR: 0.00007363   Polyak: 0.99990   Bf: 100   Loss: 6.878  0.441  2.125
Ep: 24010   Rew: -73.95   Avg Rew: -19.17   LR: 0.00007383   Polyak: 0.99990   Bf: 100   Loss: 6.018  1.431  1.878
Ep: 24020   Rew: 181.97   Avg Rew: -15.35   LR: 0.00007307   Polyak: 0.99990   Bf: 100   Loss: 6.962  0.848  1.235
Ep: 24030   Rew: -33.45   Avg Rew: -23.70   LR: 0.00007474   Polyak: 0.99990   Bf: 100   Loss: 6.774  1.563  1.007
Ep: 24040   Rew: 112.68   Avg Rew: -21.09   LR: 0.00007422   Polyak: 0.99990   Bf

Ep: 24680   Rew: -6.38   Avg Rew: -14.35   LR: 0.00007287   Polyak: 0.99990   Bf: 100   Loss: 6.409  0.426  0.540
Ep: 24690   Rew: 2.46   Avg Rew: -13.23   LR: 0.00007265   Polyak: 0.99990   Bf: 100   Loss: 7.295  1.027  1.040
Ep: 24700   Rew: -104.17   Avg Rew: -18.68   LR: 0.00007374   Polyak: 0.99990   Bf: 100   Loss: 6.273  0.730  0.559
Ep: 24710   Rew: -71.73   Avg Rew: -17.93   LR: 0.00007359   Polyak: 0.99990   Bf: 100   Loss: 6.959  0.806  1.569
Ep: 24720   Rew: -83.32   Avg Rew: -26.62   LR: 0.00007532   Polyak: 0.99990   Bf: 100   Loss: 6.331  0.387  0.686
Ep: 24730   Rew: 173.94   Avg Rew: -17.49   LR: 0.00007350   Polyak: 0.99990   Bf: 100   Loss: 6.787  0.400  0.479
Ep: 24740   Rew: -37.73   Avg Rew: -21.10   LR: 0.00007422   Polyak: 0.99990   Bf: 100   Loss: 6.432  0.976  1.591
Ep: 24750   Rew: -94.42   Avg Rew: -16.81   LR: 0.00007336   Polyak: 0.99990   Bf: 100   Loss: 6.855  0.603  0.718
Ep: 24760   Rew: -84.25   Avg Rew: -16.90   LR: 0.00007338   Polyak: 0.99990   Bf:

Ep: 25400   Rew: -69.59   Avg Rew: -16.84   LR: 0.00007337   Polyak: 0.99990   Bf: 100   Loss: 6.606  0.542  0.663
Ep: 25410   Rew: -36.10   Avg Rew: -18.11   LR: 0.00007362   Polyak: 0.99990   Bf: 100   Loss: 6.345  0.503  0.330
Ep: 25420   Rew: -87.38   Avg Rew: -17.64   LR: 0.00007353   Polyak: 0.99990   Bf: 100   Loss: 6.524  3.985  0.774
Ep: 25430   Rew: -81.04   Avg Rew: -20.83   LR: 0.00007417   Polyak: 0.99990   Bf: 100   Loss: 6.510  5.388  5.355
Ep: 25440   Rew: -87.79   Avg Rew: -21.34   LR: 0.00007427   Polyak: 0.99990   Bf: 100   Loss: 6.780  0.775  0.778
Ep: 25450   Rew: -85.07   Avg Rew: -21.86   LR: 0.00007437   Polyak: 0.99990   Bf: 100   Loss: 6.432  1.292  1.489
Ep: 25460   Rew: -76.79   Avg Rew: -22.45   LR: 0.00007449   Polyak: 0.99990   Bf: 100   Loss: 7.487  0.509  0.508
Ep: 25470   Rew: -13.24   Avg Rew: -20.66   LR: 0.00007413   Polyak: 0.99990   Bf: 100   Loss: 6.304  1.616  0.480
Ep: 25480   Rew: -0.20   Avg Rew: -19.28   LR: 0.00007386   Polyak: 0.99990   Bf

KeyboardInterrupt: 

In [ ]:
def test():  
    random_seed = 0
    n_episodes = 3
    max_timesteps = 2000
    render = True
    save_gif = True
    
    filename = "TD3_torch_{}_{}".format(env_name, random_seed)
    filename += ''
    directory = "./preTrained/td3_torch/{}".format(env_name)
    
    env = gym.make(env_name)
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.shape[0]
    max_action = float(env.action_space.high[0])
    
    policy = TD3(state_dim, action_dim, max_action)
    
    policy.load_actor(directory, filename)
    
    for ep in range(1, n_episodes+1):
        ep_reward = 0
        state = env.reset()
        for t in range(max_timesteps):
            action = policy.select_action(state)
            state, reward, done, _ = env.step(action)
            ep_reward += reward
            if render:
                env.render()
                if save_gif:
                    dirname = './gif/td3_torch/{}'.format(ep)
                    if not os.path.isdir(dirname):
                        os.mkdir(dirname)
                    img = env.render(mode = 'rgb_array')
                    img = Image.fromarray(img)
                    img.save('./gif/td3_torch/{}/{}.jpg'.format(ep,t))
            if done:
                break
            
        print('Episode: {}\tReward: {}'.format(ep, int(ep_reward)))
        ep_reward = 0
        env.close()        
                
test()
    
    